In [1]:
import pandas as pd
df=pd.read_csv("raw.csv")
df[["CP", "Q", "CPCleaned", "QCleaned"]]=df[["CP", "Q", "CPCleaned", "QCleaned"]].applymap(lambda x : x.replace("  ", " ").lower().strip())

In [6]:
filtered_df=df[df['CPCleaned']=="emergency preparedness"]
pd.crosstab(filtered_df['Country'], filtered_df['QCleaned'],margins=True).transpose().head(1)

Country,Bangladesh,Cambodia,Haiti,Indonesia,Jordan,Nicaragua,Vietnam,All
QCleaned,,,,,,,,
accessable unlocked emergency exits,584,2503,161,851,474,122,1802,6497


In [5]:
df[(df.QCleaned=="accessable unlocked emergency exits")&(df.Country=="Nicaragua")].Q.value_counts()

are the emergency exits accessible, unobstructed and unlocked during working hours, including overtime?    122
Name: Q, dtype: int64

In [45]:
with open("./stata.do", "r") as docmd:
    dol=docmd.read().split("\n")
    dol=[x for x in dol if x.startswith("//")==False and x.strip()!=""]

```stata
replace Itemscleaned="Forced overtime under threat of penalty" if regexm(Q,"workers forced to work overtime in order to earn minimum wage") 

drop if regexm(Itemscleaned,"employer delay or withhold") & Country=="Nicaragua"

drop if Itemscleaned=="Paid temporary workers correcdtly" & Country=="Haiti"

drop if Itemscleaned=="Paid piece rate workers correctluy" & Country=="Haiti"

drop if Q=="Are HIV/AIDS tests required at hiring or at any time during employment?" 

drop if Q=="Does the employer restrict workers from leaving the workplace?  " & Country=="Haiti"

drop if Q=="Has the employer communicated and disseminated laws and regulations on prevention and control of sexual harassment at the workplace to workers?" & Country=="Vietnam"
```

In [53]:
import pandas as pd
dff=pd.read_stata("d://raw.dta") #1277521
df=dff.copy()

In [54]:
for i in dol:
    stataparser(i)

In [55]:
len(df)

1265047

In [52]:
import re
def stataparser(stata_command):
    global df
    pattern_match = re.search(r'regexm\(Q,"(.*?)"\)', stata_command)
    if not pattern_match:
        raise ValueError("Invalid Stata command: missing regex pattern.")
    pattern = pattern_match.group(1)

    condition_match = re.search(r'&\s*(\w+)\s*==\s*"(.*?)"', stata_command)
    if condition_match:
        column, value = condition_match.group(1), condition_match.group(2)
        # print(2,pattern,column,value,sep="\n")
        df = df[~(df['Q'].str.contains(pattern) & (df[column] == value))]
    else:
        pass
        # print(1,pattern,sep="\n")
        df = df[~df['Q'].str.contains(pattern)]
    
    # return df

In [34]:
stataparser('drop if regexm(Q,"in advance in order to obtain the authorization before the dismissal of a pregnant worker?") ')

1
in advance in order to obtain the authorization before the dismissal of a pregnant worker?


---
# [V2](https://cornell.app.box.com/file/1564849247058?sb=/activity/versions/1720499049236) (`1.03GB`):

In [56]:
df=pd.read_stata("d://raw.dta")
df.drop(["year"],axis=1, inplace=True)
df["id"]=df.index

In [61]:
df.to_csv("raw.csv", index=False,encoding="utf-8")